In [1]:
!nvidia-smi

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"

import numpy as np
import cv2
import matplotlib.pyplot as plt
import SimpleITK as sitk
import cv2
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('/workspace/sunggu'))
sys.path.append(os.path.abspath('/workspace/sunggu/4.Dose_img2img'))
sys.path.append(os.path.abspath('/workspace/sunggu/MONAI'))
from sunggu_utils import check_value, take_list, plot_confusion_matrix, list_sort_nicely, find_dir, plot_3D

Fri Jan  8 09:05:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:02:00.0 Off |                  N/A |
| 41%   42C    P8    14W / 280W |  11866MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 41%   

# Setup imports

In [2]:
import glob
import torch
import os
import shutil
import tempfile
import matplotlib.pyplot as plt

import monai
from monai.utils import first, set_determinism
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import CacheDataset, DataLoader, Dataset
from monai.inferers import sliding_window_inference
from monai.losses import DiceLoss, GeneralizedDiceLoss, FocalLoss, TverskyLoss, NoiseRobustDiceLoss
from monai.metrics import compute_meandice, DiceMetric
from monai.transforms import (
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadNiftid,
    Lambdad,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    ToTensord,
    RandScaleIntensityd,
    RandGaussianNoised,
    RandFlipd,
    RandZoomd,
    RandGaussianNoised,
    RandGaussianSmoothd,
    RandShiftIntensityd,
    SpatialPadd,
    RandAffined,
    CastToTyped,
    DeleteItemsd,
    FgBgToIndicesd,
    Rand3DElasticd,
    RandZoomd,
    Rand2DElasticd,
    RandWeightedCropd,
    AsDiscrete,
)

print_config()

MONAI version: 0+unknown
Python version: 3.6.9 (default, Jul 17 2020, 12:50:27)  [GCC 8.4.0]
OS version: Linux (4.4.0-194-generic)
Numpy version: 1.18.5
Pytorch version: 1.6.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.2.0
scikit-image version: 0.17.2
Pillow version: 8.0.1
Tensorboard version: 2.3.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.7.0
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.50.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [3]:
import random

random_seed = 0
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

set_determinism(seed=0)

In [4]:
from sklearn.model_selection import train_test_split

def split_index(dcm_list):
    X_train, X_test = train_test_split(dcm_list, test_size=0.2, random_state=0)
    return X_train, X_test

In [5]:
low_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/dcm_original_dataset/*/*Low Dose Neck*/*dcm"))
high_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/dcm_original_dataset/*/*Neck Other*/*dcm"))

low_folder = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/dcm_original_dataset/*"))
high_folder = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/dcm_original_dataset/*"))

In [6]:
low_data_dicts = [{"image": image_name} for image_name in low_images]
high_data_dicts = [{"image": image_name} for image_name in high_images]


# 클렌징

In [ ]:
# from pydicom import dcmread
# import shutil

# for i in tqdm(low_images):
#     dcm_img = dcmread(i)
#     if ('b40f' in dcm_img[0x0008,0x103E].value.lower()) or not ('cor' in dcm_img[0x0008,0x103E].value.lower() ):
#         folder_dir = i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/').replace(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/').split('/')[-1], '')
#         if not os.path.exists(folder_dir):
#             os.makedirs(folder_dir, mode=0o777)
            
#         shutil.copy(i, i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))
        


# for i in tqdm(high_images):
#     dcm_img = dcmread(i)
#     if ('b40f' in dcm_img[0x0008,0x103E].value.lower()) or not ('cor' in dcm_img[0x0008,0x103E].value.lower() ):
#         folder_dir = i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/').replace(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/').split('/')[-1], '')
#         if not os.path.exists(folder_dir):
#             os.makedirs(folder_dir, mode=0o777)
            
#         shutil.copy(i, i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))
        

## dcm 나누기

##### 종류 2개인지 check

In [ ]:
# from pydicom import dcmread

# for folder in tqdm(low_folder):
#     desc_list = []
#     dcm_dir = glob.glob(folder+'/*Low Dose Neck*/*dcm')
#     for i in dcm_dir:
#         dcm_img = dcmread(i)
#         desc_list.append(dcm_img[0x0008,0x103E].value.lower())
        
#     if (len(np.unique(desc_list)) != 2):
#         print(len(np.unique(desc_list)))
#         print(folder)
        
# for folder in tqdm(low_folder):
#     desc_list = []
#     dcm_dir = glob.glob(folder+'/*Neck Other*/*dcm')
#     for i in dcm_dir:
#         dcm_img = dcmread(i)
#         desc_list.append(dcm_img[0x0008,0x103E].value.lower())
        
#     if (len(np.unique(desc_list)) != 2):
#         print(len(np.unique(desc_list)))
#         print(folder)
        

##### 파일 복사

In [ ]:
# from pydicom import dcmread
# import shutil

# for folder in tqdm(low_folder):
#     dcm_dir = glob.glob(folder+'/*Low Dose Neck*/*dcm')
#     cnt=0
#     for i in dcm_dir:
        
#         if not os.path.exists(os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/0/'):
#             os.makedirs(os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/0/', mode=0o777)
#         if not os.path.exists(os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/1/'):
#             os.makedirs(os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/1/', mode=0o777)

#         if (cnt==0):
#             dcm_img = dcmread(i)
#             first_kind = dcm_img[0x0008,0x103E].value.lower()    
        
#         cnt+=1
#         dcm_img = dcmread(i)
#         if (first_kind == dcm_img[0x0008,0x103E].value.lower()):
#             shutil.copy(i, os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/0/' + i.split('/')[-1])
            
#         else :
#             shutil.copy(i, os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/1/' + i.split('/')[-1])
        
        
# for folder in tqdm(low_folder):
#     dcm_dir = glob.glob(folder+'/*Neck Other*/*dcm')
#     cnt=0
#     for i in dcm_dir:
        
#         if not os.path.exists(os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/0/'):
#             os.makedirs(os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/0/', mode=0o777)
#         if not os.path.exists(os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/1/'):
#             os.makedirs(os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/1/', mode=0o777)

#         if (cnt==0):
#             dcm_img = dcmread(i)
#             first_kind = dcm_img[0x0008,0x103E].value.lower()    
        
#         cnt+=1
#         dcm_img = dcmread(i)
#         if (first_kind == dcm_img[0x0008,0x103E].value.lower()):
#             shutil.copy(i, os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/0/' + i.split('/')[-1])
            
#         else :
#             shutil.copy(i, os.path.dirname(i.replace('/dcm_original_dataset/', '/Cleansing_dcm_dataset/'))+ '/1/' + i.split('/')[-1])

##### 걸러내기

In [ ]:
# for idx, i in enumerate(glob.glob('/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/*')):
#     print(idx)                              

#     t = glob.glob(i+'/*Neck Other With Enhance/*/'+'*')
#     dcm_img = dcmread(t[40])
    
#     print(os.path.dirname(t[40]))
#     plt.imshow(dcm_img.pixel_array)
#     plt.show()

In [ ]:
# for idx, i in enumerate(glob.glob('/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/*')):
#     print(idx)                              

#     t = glob.glob(i+'/*Low Dose Neck With Enhance/*/'+'*')
#     dcm_img = dcmread(t[40])
    
#     print(os.path.dirname(t[40]))
#     plt.imshow(dcm_img.pixel_array)
#     plt.show()

# 매칭

In [ ]:
train_low_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/2D_dataset/Train/*/*Low Dose Neck*/*.npy"))
train_high_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/2D_dataset/Train/*/*Neck Other*/*.npy"))

test_low_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/2D_dataset/Test/*/*Low Dose Neck*/*.npy"))
test_high_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/2D_dataset/Test/*/*Neck Other*/*.npy"))

train_files = [{"low": low_name, "high": high_name} for low_name, high_name in zip(train_low_images, train_high_images)]
test_files = [{"low": low_name, "high": high_name} for low_name, high_name in zip(test_low_images, test_high_images)]

In [ ]:
dcm_train_low_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Train/*/*Low Dose Neck*/*/*.dcm"))
dcm_train_high_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Train/*/*Neck Other*/*/*.dcm"))

dcm_test_low_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Test/*/*Low Dose Neck*/*/*.dcm"))
dcm_test_high_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Test/*/*Neck Other*/*/*.dcm"))

dcm_train_files = [{"low": low_name, "high": high_name} for low_name, high_name in zip(dcm_train_low_images, dcm_train_high_images)]
dcm_test_files = [{"low": low_name, "high": high_name} for low_name, high_name in zip(dcm_test_low_images, dcm_test_high_images)]

In [ ]:
from pydicom import dcmread
cnt=0
for i in list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Test/*/*Low Dose Neck*/")):
    print(cnt)
    cnt+=1
    t=list_sort_nicely(glob.glob(i+'*/*'))
    dcm_img = dcmread(t[0])
    plt.imshow(dcm_img.pixel_array)
    plt.show()

# dcm to nii.gz

In [ ]:
low_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/dcm_original_dataset/*/*Low Dose Neck*/"))
high_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/dcm_original_dataset/*/*Neck Other*/"))


In [ ]:
import dicom2nifti
    
for dicom_directory in tqdm(low_images):
    output_folder = dicom_directory.replace('/dcm_original_dataset/', '/Cleansing_nii/')
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder, mode=0o777)
    
    dicom2nifti.convert_directory(dicom_directory, output_folder, compression=True, reorient=True)
    
    
for dicom_directory in tqdm(high_images):
    output_folder = dicom_directory.replace('/dcm_original_dataset/', '/Cleansing_nii/')
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder, mode=0o777)
    
    dicom2nifti.convert_directory(dicom_directory, output_folder, compression=True, reorient=True)
    

# 2d data 만들기

##### 경로

In [7]:
train_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Train_nii/*/*/*.nii.gz"))
test_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Test_nii/*/*/*.nii.gz"))

In [ ]:
train_files = [{"image": image_name} for image_name in train_images]
test_files = [{"image": image_name} for image_name in test_images]

In [ ]:
def CT_12bit_processing(x):
    # H, W, D
    x[x < -1024.0] = -1024.0
    x[x > 3072.0] = 3072.0
    
    x = (x + 1024.0) / 4095.0

    return x

In [ ]:
# CT에 맞는 Augmentation
train_transforms = Compose(
    [
        # 공통 전처리
        LoadNiftid(keys=["image"]),
        AddChanneld(keys=["image"]),
#         Spacingd(keys=["image"], pixdim=(0.669922, 0.669922, 3.0), mode=("nearest", "nearest")),
        Orientationd(keys=["image"], axcodes="PLS"),
        # 12 비트 CT shifting 전처리
        Lambdad(keys=["image"], func=CT_12bit_processing), 
    ]
)

## Check transforms in DataLoader

In [ ]:
check_ds = Dataset(data=train_files, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=1)
check_data = first(check_loader)

image = check_data["image"][0][0]
print(f"image shape: {image.shape}")

# plot the slice [:, :, 80]
plt.figure("check", (12, 6))
plt.subplot(1, 2, 1)
plt.title("image")
plt.imshow(image[:, :, 80], cmap="gray")

plt.show()

In [ ]:
import multiprocessing

# train_ds = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0, num_workers=int(multiprocessing.cpu_count()/2))
# test_ds = CacheDataset(data=test_files, transform=train_transforms, cache_rate=1.0, num_workers=int(multiprocessing.cpu_count()/2))
dcm_train_files

train_ds = Dataset(data=train_files, transform=train_transforms)

# train_ds = Dataset(data=train_files, transform=train_transforms)
test_ds = Dataset(data=test_files, transform=train_transforms)

In [ ]:
for batch in train_ds:
    print(batch['image'].min(), batch['image'].max())

## 저장할때 z축이 앞으로 (z, x, y) 형태

In [ ]:
folder_name = 'Train'
save_dir = '/workspace/sunggu/4.Dose_img2img/dataset/2D_dataset/'+ folder_name +'/'

for sample in tqdm(train_ds):
    image = sample['image'][0].transpose(2,0,1) 

    # 새폴더 만들기
    folder_save = save_dir + os.path.join(*sample['image_meta_dict']['filename_or_obj'].split('/Train_nii/')[-1].split('/')[:-1])
    if not os.path.exists(folder_save):
        os.makedirs(folder_save, mode=0o777)
    
    for idx, slice_img in enumerate(sample['image'][0].transpose(2,0,1)):
        np.save(folder_save +'/'+ sample['image_meta_dict']['filename_or_obj'].split('/')[-1].split('.')[0] + "_" + str(idx) + '.npy', slice_img)

In [ ]:
folder_name = 'Test'
save_dir = '/workspace/sunggu/4.Dose_img2img/dataset/2D_dataset/'+ folder_name +'/'

for sample in tqdm(test_ds):
    image = sample['image'][0].transpose(2,0,1) 

    # 새폴더 만들기
    folder_save = save_dir + os.path.join(*sample['image_meta_dict']['filename_or_obj'].split('/Test_nii/')[-1].split('/')[:-1])
    if not os.path.exists(folder_save):
        os.makedirs(folder_save, mode=0o777)
    
    for idx, slice_img in enumerate(sample['image'][0].transpose(2,0,1)):
        np.save(folder_save +'/'+ sample['image_meta_dict']['filename_or_obj'].split('/')[-1].split('.')[0] + "_" + str(idx) + '.npy', slice_img)

# dcm to npy

##### 경로

In [8]:
dcm_train_low_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Train/*/*Low Dose Neck*/*/*.dcm"))
dcm_train_high_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Train/*/*Neck Other*/*/*.dcm"))

dcm_test_low_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Test/*/*Low Dose Neck*/*/*.dcm"))
dcm_test_high_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Test/*/*Neck Other*/*/*.dcm"))

In [9]:
def CT_12bit_processing(x):
    # H, W, D
    x[x < -1024.0] = -1024.0
    x[x > 3072.0] = 3072.0
    
    x = (x + 1024.0) / 4095.0

    return x

## 저장할때 z축이 앞으로 (z, x, y) 형태

In [16]:
from pydicom import dcmread

folder_name = 'Train'
save_dir = '/workspace/sunggu/4.Dose_img2img/dataset/2D_dataset/'+ folder_name +'/'
cnt=0
for sample in tqdm(dcm_train_low_images+dcm_train_high_images):
    cnt+=1
    dcm_img = sitk.GetArrayFromImage(sitk.ReadImage(sample))
    
    result = CT_12bit_processing(dcm_img.astype('float'))
#     print(cnt)
    print(result.min(),"     ", result.max())

    # 새폴더 만들기
    folder_save = '/'+os.path.join(*sample.split('/')[:-2]).replace('/Cleansing_dcm_dataset/', '/2D_dataset/')
    if not os.path.exists(folder_save):
        os.makedirs(folder_save, mode=0o777)
    
    np.save(save_dir+sample.split('/Train/')[-1].replace('/0/', '/').replace('/1/', '/').replace('.dcm', '.npy'), result.squeeze())

  0%|          | 1/6213 [00:00<13:04,  7.92it/s]

0.0       0.6681318681318681


  0%|          | 3/6213 [00:00<17:26,  5.93it/s]

0.0       0.6617826617826618
0.0       0.6642246642246642


  0%|          | 4/6213 [00:00<16:27,  6.29it/s]

...

0.0       0.5714285714285714
0.0       0.5428571428571428
0.0       0.5079365079365079


100%|█████████▉| 6212/6213 [15:45<00:00,  9.74it/s]

0.0       0.528937728937729
0.0       0.5533577533577534
0.0       0.4998778998778999


100%|██████████| 6213/6213 [15:45<00:00,  6.57it/s]

0.0       0.5177045177045178


In [15]:
folder_name = 'Test'
save_dir = '/workspace/sunggu/4.Dose_img2img/dataset/2D_dataset/'+ folder_name +'/'
cnt=0
for sample in tqdm(dcm_test_low_images+dcm_test_high_images):
    cnt+=1
    dcm_img = sitk.GetArrayFromImage(sitk.ReadImage(sample))
    
    result = CT_12bit_processing(dcm_img.astype('float'))
    print(result.min(),"     ", result.max())
    
    # 새폴더 만들기
    folder_save = '/'+os.path.join(*sample.split('/')[:-2]).replace('/Cleansing_dcm_dataset/', '/2D_dataset/')
    if not os.path.exists(folder_save):
        os.makedirs(folder_save, mode=0o777)
    
    np.save(save_dir+sample.split('/Test/')[-1].replace('/0/', '/').replace('/1/', '/').replace('.dcm', '.npy'), result.squeeze())

  0%|          | 8/2053 [00:00<01:34, 21.58it/s]

0.0       0.67008547008547
0.0       0.6661782661782661
0.0       0.6642246642246642
0.0       0.6598290598290598
0.0       0.6573870573870574
0.0       0.6512820512820513
0.0       0.6512820512820513
0.0       0.6442002442002442
0.0       0.642002442002442


  1%|          | 18/2053 [00:00<01:08, 29.71it/s]

0.0       0.6383394383394383
0.0       0.6666666666666666
0.0       0.6681318681318681
0.0       0.6654456654456654
0.0       0.6625152625152625
0.0       0.6647130647130647
0.0       0.6644688644688644
0.0       0.673992673992674
0.0       0.662026862026862
0.0       0.6984126984126984
0.0       0.7032967032967034
0.0       0.6864468864468865


  1%|▏         | 30/2053 [00:00<00:53, 37.96it/s]

...

0.0       0.7172161172161172
0.0       0.653968253968254
0.0       0.6556776556776557
0.0       0.6195360195360196
0.0017094017094017094       0.49523809523809526
0.001221001221001221       0.5206349206349207
0.002442002442002442       0.5262515262515263
0.002442002442002442       0.5194139194139195
0.002197802197802198       0.5194139194139195
0.0014652014652014652       0.48547008547008547
0.0019536019536019536       0.5194139194139195
0.0004884004884004884       0.49474969474969477
0.0004884004884004884       0.5255189255189255


100%|█████████▉| 2047/2053 [03:42<00:00, 16.86it/s]

0.0       0.5548229548229549
0.0       0.5274725274725275
0.0       0.5382173382173382
0.0       0.5633699633699634
0.0       0.9816849816849816


100%|██████████| 2053/2053 [03:42<00:00,  9.21it/s]

0.0       1.0


# 3d data 만들기

##### 경로

In [ ]:
normal_images = list_sort_nicely(glob.glob("/workspace/sunggu/Pancreas/dataset/Train_nii/*normal_image.nii.gz"))
normal_labels = list_sort_nicely(glob.glob("/workspace/sunggu/Pancreas/dataset/Train_nii/*normal_seg.nii.gz"))


In [ ]:
train_data_dicts = [{"image": image_name, "label": label_name} for image_name, label_name in zip(train_images_list, train_labels_list)]
train_files = train_data_dicts


## Augmentation

In [ ]:
# CT에 맞는 Augmentation
train_transforms = Compose(
    [
        # 공통 전처리        
        LoadNiftid(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        Spacingd(keys=["image", "label"], pixdim=(0.669922, 0.669922), mode=("bilinear", "nearest")),
        Spacingd(keys=["image", "label"], pixdim=(0.669922, 0.669922, 3.0), mode=("nearest", "nearest")),
        Orientationd(keys=["image", "label"], axcodes="PLS"),
        
        # Intensity Normalize   [CT의 경우, Windowing 으로 해준다.]
        ScaleIntensityRanged(keys=["image"], a_min=-100.0, a_max=200.0, b_min=0.0, b_max=1.0, clip=True), 

    ]
)


## Check transforms in DataLoader

In [ ]:
check_ds = Dataset(data=train_files, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=1)
check_data = first(check_loader)

image, label = (check_data["image"][0][0], check_data["label"][0][0])
print(f"image shape: {image.shape}, label shape: {label.shape}")
# plot the slice [:, :, 80]
plt.figure("check", (12, 6))
plt.subplot(1, 2, 1)
plt.title("image")
plt.imshow(image[:, :, 80], cmap="gray")
plt.subplot(1, 2, 2)
plt.title("label")
plt.imshow(label[:, :, 80])
plt.show()

In [ ]:
import multiprocessing
from torchsampler.imbalanced import ImbalancedDatasetSampler, sunggu_ImbalancedDatasetSampler

# cf) use batch_size=2 to load images and use RandCropByPosNegLabeld
print("CPU 갯수 = ", multiprocessing.cpu_count())

# Cachedataset 이거 뭔가 문제가 있음...
train_ds = Dataset(data=train_files, transform=train_transforms)

In [ ]:
for target in tqdm(train_ds):
    img = target['image']
    mask = target['label']
    
    np.save('/workspace/sunggu/Pancreas/dataset/Train_npy/'+target['image_meta_dict']['filename_or_obj'].split('/')[-1].split('.')[0]+'.npy', img[0])
    np.save('/workspace/sunggu/Pancreas/dataset/Train_npy/'+target['label_meta_dict']['filename_or_obj'].split('/')[-1].split('.')[0]+'.npy', mask[0])